In [2]:
!pip install weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully

In [3]:
!pip install FlashRank

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.8 MB/s eta 0:00:00


In [4]:
!pip install langchain langchain_community langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [5]:
import json
import re
import uuid
from collections import defaultdict
from operator import itemgetter
from typing import Any

import pandas as pd
import weaviate
from flashrank import Ranker
from langchain_text_splitters import MarkdownHeaderTextSplitter

from langchain.retrievers import ContextualCompressionRetriever, MultiVectorRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter
from langchain.storage import InMemoryByteStore
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.vectorstores.weaviate import Weaviate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import Runnable, RunnableLambda, RunnableParallel
from langchain_text_splitters import RecursiveCharacterTextSplitter
from weaviate import EmbeddedOptions





In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from pydantic.v1 import BaseSettings, Field
from pathlib  import Path

In [8]:
DOCUMENT_TEMPLATE = """Тематика: {title}

Текст: {text}

"""

In [9]:
QUESTION_ANSWER_TEMPLATE = """Вы являетесь помощником при выполнении заданий по поиску ответов на \
вопросы.
Используйте приведенные ниже фрагменты из извлеченного контекста, чтобы ответить на вопрос.
Если вы не знаете ответа, просто предложите обатиться в поддержку пользователей.

Вопрос: {question}
Контекст: {context}
Ответ: """



INPUT_ANSWER_TEMPLATE = """Вы являетесь помощником при выполнении заданий по поиску ответов на \
вопросы.
Используйте приведенные ниже фрагменты из извлеченного контекста, чтобы ответить на вопрос.
!!! Если вы не знаете ответа, просто скажите, предложите обатиться в поддержку пользователей.

Вопрос: {input}
Контекст: {context}
Ответ: """



CHAIN_FILTER_TEMPLATE = """Учитывая следующий вопрос и контекст, верните YES, если контекст \
относится к вопросу, и NO, если это не так.

 Вопрос: {question}
 Контекст:
{context}

 Релевантно (YES / NO): """

In [ ]:
from typing import Optional, Sequence

from flashrank import Ranker, RerankRequest
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor
from langchain_core.callbacks.manager import Callbacks
from langchain_core.documents import Document
from pydantic import Extra

class LocalFlashrankRerank(BaseDocumentCompressor):
    """Document compressor using Flashrank interface."""
    client: Ranker
    """Flashrank client to use for compressing documents"""
    top_n: int = 3
    """Number of documents to return."""
    model: Optional[str] = None
    """Model to use for reranking."""

    class Config:
        """Configuration for this pydantic object."""
        extra = Extra.forbid
        arbitrary_types_allowed = True

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        passages = [
            {"id": i, "text": doc.page_content, "metadata": doc.metadata}
            for i, doc in enumerate(documents)
        ]

        rerank_request = RerankRequest(query=query, passages=passages)
        rerank_response = self.client.rerank(rerank_request)[: self.top_n]
        final_results = []

        for r in rerank_response:
            doc = Document(
                page_content=r["text"],
                metadata={"id": r["id"], "relevance_score": r["score"]} | r["metadata"],
            )
            final_results.append(doc)

        return final_results

In [ ]:
import os
from datetime import datetime, timedelta, tzinfo
from typing import Any, Optional
from uuid import UUID
from langchain.schema import LLMResult
from langchain_core.callbacks import BaseCallbackHandler
from tqdm.auto import tqdm


class BatchCallback(BaseCallbackHandler):
    def __init__(self, total: int):
        super().__init__()
        self.count = 0
        self.progress_bar = tqdm(total=total)

    def on_llm_end(
        self,
        response: LLMResult,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> Any:
        self.count += 1
        self.progress_bar.update(1)

def preprocess_timestamp(timestamp: datetime, source_timezone: tzinfo, target_timezone) -> datetime:
    output_timestamp = timestamp

    if timestamp and source_timezone:
        output_timestamp = output_timestamp.replace(tzinfo=source_timezone)

        if target_timezone:
            output_timestamp = output_timestamp.astimezone(target_timezone)

    return output_timestamp

In [10]:
class DefaultSettings(BaseSettings):
  project_path: Path = Path("/content/drive/MyDrive/rutube-hack")
  data_path: Path = project_path / "data"
  raw_data_path: Path = data_path / "raw"
  db_collection_name: str = "Rutube"
  vectorstore_data_path: Path = data_path / "vectorstore"

In [11]:
settings = DefaultSettings()

In [12]:
def load_data(settings: DefaultSettings) -> pd.DataFrame:
  raw_data = []

  for file_path in settings.raw_data_path.glob("*.md"):
    # print(file_path)
    # print(type(file_path))
    # print(file_path.name)
    with open(file_path, encoding="utf-8") as f:
      try:
        md_data = f.read()
      except Exception as e:
        print(f"{file_path.name}: {e}")
        raise
      # print(md_data)

      dict_from_md_data = {"text": md_data, "file_name": file_path.name}
      # md_data["file_name"] = file_path.name
      raw_data.append(dict_from_md_data)

  raw_df = pd.DataFrame(raw_data)
  raw_df = raw_df.rename(lambda x: x.lower(), axis=1)
  # raw_df = raw_df.rename({"title": "name"}, axis=1)

  return raw_df

In [13]:
raw_df = load_data(settings)

In [14]:
raw_df

,text,file_name
0,# Встраивание видео с RUTUBE\n\n## Содержание\...,FAQ_ Встраивание (embed) видео.md
1,**ЛИЦЕНЗИОННОЕ СОГЛАШЕНИЕ НА ИСПОЛЬЗОВАНИЕ ПРО...,ЛИЦЕНЗИОННОЕ СОГЛАШЕНИЕ НА ИСПОЛЬЗОВАНИЕ ПЛЕЕР...
2,** УСЛОВИЯ РАЗМЕЩЕНИЯ КОНТЕНТА:**\n\n1. ...,Условия размещения контента.md
3,"# РЕГИСТРАЦИЯ, ЛИЧНЫЕ ДАННЫЕ, АККАУНТ\n\n## За...",FAQ_ Часто задаваемые вопросы RUTUBE.md
4,# Shorts на RUTUBE\n\nShorts — короткие вертик...,FAQ_ Shorts.md
5,# Монетизация на RUTUBE\n\nМонетизацию можно п...,FAQ_ Монетизация на RUTUBE.md
6,# СОГЛАСИЕ НА НАПРАВЛЕНИЕ РЕКЛАМНЫХ СООБЩЕНИЙ ...,Согласие на направление рекламных сообщений и...
7,# **Политика Cookies:**\n\n**ИНФОРМАЦИЯ ОБ ИСП...,Политика COOKIES сайта RUTUBE.md
8,# **ПОЛИТИКА КОНФИДЕНЦИАЛЬНОСТИ И ОБРАБОТКИ ПЕ...,Политика конфиденциальности и обработки персон...
9,## Что такое трансляции (стримы) на RUTUBE?\n\...,FAQ_ Трансляции на RUTUBE.md


In [15]:
raw_df.loc[0, 'file_name']

'FAQ_ Встраивание (embed) видео.md'

In [16]:
headers_to_split_on = [
    ("#", "Header_1"),
    ("##", "Header_2"),
    ("###", "Header_3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = [markdown_splitter.split_text(document) for document in raw_df["text"]]

In [17]:
chunk_data = []
for i in range(len(splits)):
  for chunk_content in splits[i]:
    chunk_i_dict = {'text': chunk_content.page_content, 'file_name': raw_df.loc[i, 'file_name']}
    for header_name in ['Header_1', 'Header_2', 'Header_3']:
      try:
        chunk_i_dict[header_name] = chunk_content.metadata[header_name]
      except Exception:
        chunk_i_dict[header_name] = None

    chunk_data.append(chunk_i_dict)

In [18]:
chunk_df = pd.DataFrame(chunk_data)
chunk_df

,text,file_name,Header_1,Header_2,Header_3
0,[Что такое встраивание (embed) видео?](https:/...,FAQ_ Встраивание (embed) видео.md,Встраивание видео с RUTUBE,Содержание,None
1,"Эмбед или эмбедирование — функция, которая поз...",FAQ_ Встраивание (embed) видео.md,Встраивание видео с RUTUBE,Что такое встраивание (embed) видео?,None
2,Чтобы встроить (эмбедировать) видео с RUTUBE н...,FAQ_ Встраивание (embed) видео.md,Встраивание видео с RUTUBE,Как эмбедировать видео с RUTUBE?,None
3,"Чтобы изменить размер окна плеера, цвет элемен...",FAQ_ Встраивание (embed) видео.md,Встраивание видео с RUTUBE,Как эмбедировать видео с RUTUBE?,Дополнительные возможности
4,После вставки кода видео в код страницы в него...,FAQ_ Встраивание (embed) видео.md,Встраивание видео с RUTUBE,Как эмбедировать видео с RUTUBE?,Настройки параметров в коде вставки
...,...,...,...,...,...
279,RUTUBE? \nДо 5 устройств Smart TV.,FAQ_ RUTUBE для Smart TV.md,None,Сколько телевизоров или ТВ-приставок может быт...,None
280,входа? \nПерейдите по ссылке <http://rutube.r...,FAQ_ RUTUBE для Smart TV.md,None,"Как войти в профиль, если при сканировании QR-...",None
281,доступен только на сайте rutube.ru»? \nЭто ош...,FAQ_ RUTUBE для Smart TV.md,None,"Что делать, если видео недоступно только на ТВ...",None
282,делать? \nНекоторые прямые эфиры временно нед...,FAQ_ RUTUBE для Smart TV.md,None,В приложении RUTUBE для ТВ недоступны каналы С...,None


In [19]:
from langchain_community.embeddings import CohereEmbeddings
from google.colab import userdata


In [47]:
def build_vectorstore(documents_df: pd.DataFrame, settings: DefaultSettings) -> Weaviate:
    client = weaviate.Client(
        embedded_options=EmbeddedOptions(persistence_data_path=settings.vectorstore_data_path)
    )

    if client.schema.exists(settings.db_collection_name):
        client.schema.delete_class(settings.db_collection_name)
    class_obj = {"class": settings.db_collection_name, "vectorIndexConfig": {"distance": "cosine"}}
    client.schema.create_class(class_obj)

    embeddings_model = CohereEmbeddings(model="embed-multilingual-v3.0", truncate="NONE", cohere_api_key=userdata.get('cohere'), user_agent="langchain")

    texts = documents_df["text"].tolist()
    metadatas = documents_df[["file_name", "Header_1", "Header_2", "Header_3"]].to_dict(orient="records")

    chunk_data = []
    for i in range(len(splits)):
      for chunk_content in splits[i]:
        chunk_i_dict = {'text': chunk_content.page_content, 'file_name': raw_df.loc[i, 'file_name']}
        for header_name in ['Header_1', 'Header_2', 'Header_3']:
          try:
            chunk_i_dict[header_name] = chunk_content.metadata[header_name]
          except Exception:
            chunk_i_dict[header_name] = None

        chunk_data.append(chunk_i_dict)
    df = pd.DataFrame(chunk_data)


    db = Weaviate.from_texts(
        texts=[text[:1400] for text in texts],
        embedding=embeddings_model,
        metadatas=metadatas,
        client=client,
        index_name=settings.db_collection_name,
    )

    response = client.query.aggregate(settings.db_collection_name).with_meta_count().do()
    print(f"Number of documents in database: {response}")

    return db

In [49]:
def prepare_db(settings: DefaultSettings) -> Weaviate:
    raw_df = load_data(settings)

    db = build_vectorstore(raw_df, settings)
    return db

In [50]:
def build_parent_docs_store():
    store = InMemoryByteStore()
    return store

In [ ]:
def build_child_docs_vectorstore(settings: DefaultSettings) -> Weaviate:
    client = weaviate.Client(
        embedded_options=EmbeddedOptions(persistence_data_path=settings.vectorstore_data_path)
    )

    db_collection_name = "Rutube"

    if client.schema.exists(db_collection_name):
        client.schema.delete_class(db_collection_name)

    class_obj = {"class": db_collection_name, "vectorIndexConfig": {"distance": "cosine"}}
    client.schema.create_class(class_obj)

    embeddings_model = CohereEmbeddings(model="embed-multilingual-v3.0", truncate="NONE", cohere_api_key=userdata.get('cohere'), user_agent="langchain")

    db = Weaviate(
        client=client,
        index_name=db_collection_name,
        text_key="text",
        embedding=embeddings_model,
        attributes=["doc_id", "file_name"],
        by_text=False,
    )

    return db

In [ ]:
def prepare_multivector_retriever(documents_df, *, model, settings: DefaultSettings):
    store = build_parent_docs_store()
    vectorstore = build_child_docs_vectorstore(settings)

    texts = documents_df["text"].tolist()
    metadatas = documents_df[["file_name"]].to_dict(orient="records")
    doc_ids = [str(uuid.uuid4()) for _ in texts]

    id_key = "doc_id"
    documents = [
        Document(page_content=text, metadata={id_key: doc_id} | metadata)
        for text, metadata, doc_id in zip(texts, metadatas, doc_ids)
    ]
    child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512, chunk_overlap=0, length_function=model.get_num_tokens
    )
    docs = child_splitter.split_documents(documents)

    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        byte_store=store,
        id_key=id_key,
    )  # type: ignore
    retriever.vectorstore.add_documents(docs)
    retriever.docstore.mset(list(zip(doc_ids, documents)))

    child_docs_count = (
        retriever.vectorstore._client.query.aggregate(settings.db_collection_name)  # type: ignore
        .with_meta_count()
        .do()
    )
    parent_docs_count = len(retriever.docstore.mget(doc_ids))
    print(f"Parent documents count: {parent_docs_count}")
    print(f"Child documents count: {child_docs_count}")

    return retriever

In [ ]:
def generate_answer(question: str, chain: Runnable) -> dict[str, Any]:
  response = chain.invoke({"question": question})
  return response

In [26]:
def generate_answers(questions: list[str], chain: Runnable) -> list[dict[str, Any]]:
    callback = BatchCallback(len(questions))
    response = chain.batch(
        [{"question": question} for question in questions],
        config={"max_concurrency": 5, "callbacks": [callback]},
    )
    return response

In [27]:
# def preprocess_file_name(text, file_extension):
    # return re.search(rf"(.*?).{file_extension}", text).group(1)

In [28]:
# def preprocess_json_data(texts):
#     return list(map(preprocess_file_name, texts, file_extension="json"))

In [29]:
# def find_position(reference_answer_source, llm_answer_sources):
#     return next(
#         (i for i, x in enumerate(llm_answer_sources, start=1) if x == reference_answer_source), -1
#     )

In [30]:
# def answer_has_reference_chunk(chunk_position_number):
#     return int(chunk_position_number != -1)

In [31]:
def get_llm_answers_info(llm_answers):

    llm_answers_info = defaultdict(list)

    for answer_with_sources in llm_answers:
        llm_answers_info["llm_answer"].append(answer_with_sources["answer"])
        llm_answers_info["llm_context"].append(
            "\n\n".join(
                [
                    f"Chunk {i+1}:\n" + answer_with_sources["context"][i].page_content
                    for i in range(len(answer_with_sources["context"]))
                ]
            )
        )
        # llm_answers_info["llm_answer_source"].append(
        #     [
        #         (
        #             preprocess_file_name(
        #                 answer_with_sources["context"][i].metadata["file_name"],
        #                 file_extension="json",
        #             ),
        #             answer_with_sources["context"][i].metadata["page_num"],
        #         )
        #         for i in range(len(answer_with_sources["context"]))
        #     ]
        # )
        # if "all_documents" in answer_with_sources:
        #     llm_answers_info["all_sources"].append(
        #         [
        #             (
        #                 preprocess_file_name(
        #                     answer_with_sources["all_documents"][i].metadata["file_name"],
        #                     file_extension="json",
        #                 ),
        #                 answer_with_sources["all_documents"][i].metadata["page_num"],
        #             )
        #             for i in range(len(answer_with_sources["all_documents"]))
        #         ]
        #     )
    return llm_answers_info

In [32]:
# def find_reference_answer_chunk_position(df, llm_answer_sources):
#     reference_answer_sources = list(zip(df["file_name"]))
#     reference_answer_chunk_position = [
#         find_position(reference_answer_source, llm_answer_source)
#         for reference_answer_source, llm_answer_source in zip(
#             reference_answer_sources, llm_answer_sources
#         )
#     ]
#     return reference_answer_chunk_position

In [33]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [34]:
def create_chain(db: Weaviate, settings: DefaultSettings) -> Runnable:

    response = db._client.query.aggregate(settings.db_collection_name).with_meta_count().do()
    rows_count = response["data"]["Aggregate"][settings.db_collection_name][0]["meta"]["count"]

    all_documents_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": rows_count},
    )

    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 10},
    )

    # retriever = db.as_retriever(
    #     search_type="mmr",
    #     search_kwargs={
    #         "k": 10,
    #         "fetch_k": 25,
    #         "lambda_mult": 0.5,
    #     },
    # )

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    model = ### MODEL
    output_parser = StrOutputParser()

    chain_without_source = prompt | model | output_parser
    map_params: dict[str, Any] = {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "all_documents": itemgetter("question") | all_documents_retriever,
    }
    chain_with_source = RunnableParallel(map_params).assign(answer=chain_without_source)

    return chain_with_source

In [35]:
def create_chain_with_reranking(db: Weaviate, settings: DefaultSettings) -> Runnable:
    # response = db._client.query.aggregate(settings.db_collection_name).with_meta_count().do()
    # rows_count = response["data"]["Aggregate"][settings.db_collection_name][0]["meta"]["count"]
    # all_documents_base_retriever = db.as_retriever(
    #     search_type="similarity",
    #     search_kwargs={"k": rows_count},
    # )
    base_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 25},
    )

    reranker_model_name = "ms-marco-MultiBERT-L-12"
    client = Ranker(model_name=reranker_model_name, cache_dir=settings.models_path)
    compressor = LocalFlashrankRerank(top_n=10, client=client, model=reranker_model_name)
    retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=base_retriever
    )
    # all_documents_retriever = ContextualCompressionRetriever(
    #     base_compressor=compressor, base_retriever=all_documents_base_retriever
    # )

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    model = ### MODEL

    output_parser = StrOutputParser()

    chain_without_source = prompt | model | output_parser
    map_params: dict[str, Any] = {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        # "all_documents": itemgetter("question") | all_documents_retriever,
    }
    chain_with_source = RunnableParallel(map_params).assign(answer=chain_without_source)

    return chain_with_source

In [36]:
def create_chain_with_doc_formatting(db: Weaviate, settings: DefaultSettings) -> Runnable:
    response = db._client.query.aggregate(settings.db_collection_name).with_meta_count().do()
    rows_count = response["data"]["Aggregate"][settings.db_collection_name][0]["meta"]["count"]
    all_documents_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": rows_count},
    )

    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 10},
    )

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    model = ###MODEL

    output_parser = StrOutputParser()

    map_params: dict[str, Any] = {
        "question": itemgetter("question"),
        "context": itemgetter("context") | RunnableLambda(format_docs),
    }
    chain_without_source: Runnable = RunnableParallel(map_params) | prompt | model | output_parser

    source_map_params: dict[str, Any] = {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "all_documents": itemgetter("question") | all_documents_retriever,
    }
    chain_with_source = RunnableParallel(source_map_params).assign(answer=chain_without_source)

    return chain_with_source

In [37]:
def create_chain_with_chain_filter(db: Weaviate, settings: DefaultSettings) -> Runnable:

    base_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 10},
    )

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    model = ### MODEL

    chain_filter_prompt = PromptTemplate.from_template(CHAIN_FILTER_TEMPLATE)
    chain_filter = LLMChainFilter.from_llm(model, chain_filter_prompt)
    retriever = ContextualCompressionRetriever(
        base_compressor=chain_filter, base_retriever=base_retriever
    )
    output_parser = StrOutputParser()
    chain_without_source = prompt | model | output_parser
    map_params: dict[str, Any] = {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    chain_with_source = RunnableParallel(map_params).assign(answer=chain_without_source)

    return chain_with_source

In [38]:
def create_chain_with_chain_filter_from_retriever(
    base_retriever, settings: DefaultSettings
) -> Runnable:

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    model = ### MODEL
    chain_filter_prompt = PromptTemplate.from_template(CHAIN_FILTER_TEMPLATE)
    chain_filter = LLMChainFilter.from_llm(model, chain_filter_prompt)
    retriever = ContextualCompressionRetriever(
        base_compressor=chain_filter, base_retriever=base_retriever
    )
    output_parser = StrOutputParser()
    chain_without_source = prompt | model | output_parser
    map_params: dict[str, Any] = {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    chain_with_source = RunnableParallel(map_params).assign(answer=chain_without_source)

    return chain_with_source

In [39]:
def create_multivector_chain(retriever: MultiVectorRetriever, model: GigaChat) -> Runnable:

    db = retriever.vectorstore
    query = db._client.query  # type: ignore
    response = query.aggregate(settings.db_collection_name).with_meta_count().do()
    rows_count = response["data"]["Aggregate"][settings.db_collection_name][0]["meta"]["count"]

    all_documents_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": rows_count},
    )

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    output_parser = StrOutputParser()

    map_params: dict[str, Any] = {
        "question": itemgetter("question"),
        "context": itemgetter("context") | RunnableLambda(format_docs),
    }
    chain_without_source: Runnable = RunnableParallel(map_params) | prompt | model | output_parser
    source_map_params: dict[str, Any] = {
        "context": itemgetter("question")
        | all_documents_retriever
        | RunnableLambda(lambda x: x[:10]),
        "question": itemgetter("question"),
        "all_documents": itemgetter("question") | all_documents_retriever,
    }
    chain_with_source = RunnableParallel(source_map_params).assign(answer=chain_without_source)

    return chain_with_source

In [40]:
def create_multivector_with_compressor_chain(
    retriever: MultiVectorRetriever, model: GigaChat
) -> Runnable:
    db = retriever.vectorstore
    query = db._client.query  # type: ignore
    response = query.aggregate(settings.db_collection_name).with_meta_count().do()
    rows_count = response["data"]["Aggregate"][settings.db_collection_name][0]["meta"]["count"]
    all_documents_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": rows_count},
    )
    compressor = LLMChainExtractor.from_llm(model)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=all_documents_retriever,
    )

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    output_parser = StrOutputParser()

    map_params: dict[str, Any] = {
        "question": itemgetter("question"),
        "context": itemgetter("context") | RunnableLambda(format_docs),
    }
    chain_without_source: Runnable = RunnableParallel(map_params) | prompt | model | output_parser
    source_map_params: dict[str, Any] = {
        "context": itemgetter("question")
        | compression_retriever
        | RunnableLambda(lambda x: x[:10]),
        "question": itemgetter("question"),
        "all_documents": itemgetter("question") | compression_retriever,
    }
    chain_with_source = RunnableParallel(source_map_params).assign(answer=chain_without_source)

    return chain_with_source

In [41]:
def run(settings: DefaultSettings) -> None:
    db = prepare_db(settings)

    chain = create_chain(db, settings)

    question = "Зачем нужно хеджирование?"
    response = generate_answer(question, chain)
    print(response)

In [42]:
def run_multivector(settings: DefaultSettings):
    raw_df = load_data(settings)
    embeddings_model = CohereEmbeddings(model="embed-multilingual-v3.0", truncate="NONE", cohere_api_key=userdata.get('cohere'), user_agent="langchain")

    texts = raw_df["text"].tolist()
    metadatas = raw_df[["file_name", "Header_1", "Header_2", "Header_3"]].to_dict(orient="records")

    chunk_data = []
    for i in range(len(splits)):
      for chunk_content in splits[i]:
        chunk_i_dict = {'text': chunk_content.page_content, 'file_name': raw_df.loc[i, 'file_name']}
        for header_name in ['Header_1', 'Header_2', 'Header_3']:
          try:
            chunk_i_dict[header_name] = chunk_content.metadata[header_name]
          except Exception:
            chunk_i_dict[header_name] = None

        chunk_data.append(chunk_i_dict)
    df = pd.DataFrame(chunk_data)


    model = ### MODEL
    retriever = prepare_multivector_retriever(df, model=model, settings=settings)
    chain = create_multivector_chain(retriever, model)

    question = (
        "Опиши стратегию «Коллар» и его использование "
        "для хеджирования базового актива на примере нефти"
    )
    response = generate_answer(question, chain)
    print(response)


In [43]:
def run_batch(settings: DefaultSettings) -> None:
    db = prepare_db(settings)

    # chain = create_chain(db, settings)
    # chain = create_chain_with_reranking(db, settings)
    chain = create_chain_with_doc_formatting(db, settings)
    # chain = create_chain_with_chain_filter(db, settings)

    questions_with_answers_df = pd.read_excel(
        settings.external_data_path / "Вопросы.xlsx",
        names=["file_name", "name", "page_num", "question", "answer"],
    )
    questions_with_answers_df["file_name"] = questions_with_answers_df["file_name"].str.normalize(
        "NFD"
    )
    questions_with_answers = questions_with_answers_df["question"].tolist()
    responses = generate_answers(questions_with_answers, chain)
    llm_answers_info = get_llm_answers_info(responses)
    # questions_with_answers_df["llm_answer"] = llm_answers_info["llm_answer"]
    questions_with_answers_df["llm_context"] = llm_answers_info["llm_context"]
    # questions_with_answers_df["file_name"] = questions_with_answers_df["file_name"].apply(
    #     lambda file_name: preprocess_file_name(file_name, file_extension="pptx")
    # )

    questions_with_answers_cols = set(questions_with_answers_df.columns)

    # if "page_num" in questions_with_answers_cols:
    #     if "all_sources" in llm_answers_info:
    #         questions_with_answers_df["reference_chunk_position_in_all_sources"] = (
    #             find_reference_answer_chunk_position(
    #                 questions_with_answers_df, llm_answers_info["all_sources"]
    #             )
    #         )

    #     questions_with_answers_df["reference_chunk_position"] = (
    #         find_reference_answer_chunk_position(
    #             questions_with_answers_df, llm_answers_info["llm_answer_source"]
    #         )
    #     )
    #     questions_with_answers_df["found_reference_chunk"] = questions_with_answers_df[
    #         "reference_chunk_position"
    #     ].apply(answer_has_reference_chunk)
    #     retrieval_accuracy = int(
    #         round(questions_with_answers_df["found_reference_chunk"].mean() * 100)
    #     )
    #     print(f"Retrieval accuracy: {retrieval_accuracy}%")
    #     questions_with_answers_df = questions_with_answers_df.reset_index(drop=True).reindex(
    #         range(len(questions_with_answers_df) + 1)
    #     )
    #     questions_with_answers_df.loc[
    #         len(questions_with_answers_df) - 1, "found_reference_chunk"
    #     ] = f"Retrieval accuracy: {retrieval_accuracy}%"

    answers_file_path = settings.processed_data_path / "answers_for_questions_with_answers.xlsx"
    questions_with_answers_df.to_excel(answers_file_path, index=False)

    # questions_without_answers_df = pd.read_excel(
    #     settings.external_data_path / "Ограничители.xlsx",
    #     names=["question", "answer"],
    # )
    # questions_without_answers_df = questions_without_answers_df.dropna()
    # questions_without_answers = questions_without_answers_df["question"].tolist()
    # responses = generate_answers(questions_without_answers, chain)
    # questions_without_answers_df["llm_answer"] = [response["answer"] for response in responses]
    # # questions_without_answers_df["llm_context"] = [response["context"] for response in responses]
    # answers_file_path = settings.processed_data_path / "answers_for_questions_without_answers.xlsx"
    # questions_without_answers_df.to_excel(answers_file_path, index=False)

In [44]:
def create_chain_with_compressor_from_retriever(
    base_retriever, settings: DefaultSettings
) -> Runnable:

    model = ### MODEL
    compressor = LLMChainExtractor.from_llm(model)

    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever,
    )

    prompt = ChatPromptTemplate.from_messages(
        ("user", QUESTION_ANSWER_TEMPLATE),
    )

    # chain_filter_prompt = PromptTemplate.from_template(CHAIN_FILTER_TEMPLATE)
    # chain_filter = LLMChainFilter.from_llm(model, chain_filter_prompt)
    # retriever = ContextualCompressionRetriever(
    #     base_compressor=chain_filter, base_retriever=base_retriever
    # )

    output_parser = StrOutputParser()
    chain_without_source = prompt | model | output_parser
    map_params: dict[str, Any] = {
        "context": itemgetter("question") | compression_retriever,
        "question": itemgetter("question"),
    }
    chain_with_source = RunnableParallel(map_params).assign(answer=chain_without_source)

    return chain_with_source

In [45]:
def run_multivector_with_compressor_batch(settings: DefaultSettings):

    raw_df = load_data(settings)
    texts = raw_df["text"].tolist()
    metadatas = raw_df[["file_name", "Header_1", "Header_2", "Header_3"]].to_dict(orient="records")

    chunk_data = []
    for i in range(len(splits)):
      for chunk_content in splits[i]:
        chunk_i_dict = {'text': chunk_content.page_content, 'file_name': raw_df.loc[i, 'file_name']}
        for header_name in ['Header_1', 'Header_2', 'Header_3']:
          try:
            chunk_i_dict[header_name] = chunk_content.metadata[header_name]
          except Exception:
            chunk_i_dict[header_name] = None

        chunk_data.append(chunk_i_dict)
    df = pd.DataFrame(chunk_data)

    model = ### MODEL
    retriever = prepare_multivector_retriever(df, model=model, settings=settings)
    chain = create_chain_with_compressor_from_retriever(retriever, settings)
    # chain = create_chain_with_chain_filter_from_retriever(retriever, settings)

    questions_with_answers_df = pd.read_excel(
        settings.external_data_path / "Questions_20240517.xlsx",
        names=["file_name", "name", "page_num", "question", "answer"],
    )
    questions_with_answers_df["file_name"] = questions_with_answers_df["file_name"].str.normalize(
        "NFD"
    )
    questions_with_answers = questions_with_answers_df["question"].tolist()
    responses = generate_answers(questions_with_answers, chain)
    llm_answers_info = get_llm_answers_info(responses)
    questions_with_answers_df["llm_answer"] = llm_answers_info["llm_answer"]
    questions_with_answers_df["llm_context"] = llm_answers_info["llm_context"]
    # questions_with_answers_df["file_name"] = questions_with_answers_df["file_name"].apply(
    #     lambda file_name: preprocess_file_name(file_name, file_extension="pptx")
    # )

    # questions_with_answers_cols = set(questions_with_answers_df.columns)

    # if "page_num" in questions_with_answers_cols:
    #     if "all_sources" in llm_answers_info:
    #         questions_with_answers_df["reference_chunk_position_in_all_sources"] = (
    #             find_reference_answer_chunk_position(
    #                 questions_with_answers_df, llm_answers_info["all_sources"]
    #             )
    #         )

    #     questions_with_answers_df["reference_chunk_position"] = (
    #         find_reference_answer_chunk_position(
    #             questions_with_answers_df, llm_answers_info["llm_answer_source"]
    #         )
    #     )
    #     questions_with_answers_df["found_reference_chunk"] = questions_with_answers_df[
    #         "reference_chunk_position"
    #     ].apply(answer_has_reference_chunk)

    #     retrieval_accuracy = int(
    #         round(questions_with_answers_df["found_reference_chunk"].mean() * 100)
    #     )
    #     print(f"Retrieval accuracy: {retrieval_accuracy}%")
    #     questions_with_answers_df = questions_with_answers_df.reset_index(drop=True).reindex(
    #         range(len(questions_with_answers_df) + 1)
    #     )

    #     questions_with_answers_df.loc[
    #         len(questions_with_answers_df) - 1, "found_reference_chunk"
    #     ] = f"Retrieval accuracy: {retrieval_accuracy}%"

    answers_file_path = settings.processed_data_path / "answers_for_questions_with_answers.xlsx"
    questions_with_answers_df.to_excel(answers_file_path, index=False)

    # questions_without_answers_df = pd.read_excel(
    #     settings.external_data_path / "Ограничители.xlsx",
    #     names=["question", "answer"],
    # )

    # questions_without_answers_df = questions_without_answers_df.dropna()
    # questions_without_answers = questions_without_answers_df["question"].tolist()
    # responses = generate_answers(questions_without_answers, chain)
    # questions_without_answers_df["llm_answer"] = [response["answer"] for response in responses]
    # questions_without_answers_df["llm_context"] = [response["context"] for response in responses]
    # answers_file_path = settings.processed_data_path / "answers_for_questions_without_answers.xlsx"
    # questions_without_answers_df.to_excel(answers_file_path, index=False)

In [46]:

def run_multivector_batch(settings: DefaultSettings):
    raw_df = load_data(settings)
    texts = raw_df["text"].tolist()
    metadatas = raw_df[["file_name", "Header_1", "Header_2", "Header_3"]].to_dict(orient="records")

    chunk_data = []
    for i in range(len(splits)):
      for chunk_content in splits[i]:
        chunk_i_dict = {'text': chunk_content.page_content, 'file_name': raw_df.loc[i, 'file_name']}
        for header_name in ['Header_1', 'Header_2', 'Header_3']:
          try:
            chunk_i_dict[header_name] = chunk_content.metadata[header_name]
          except Exception:
            chunk_i_dict[header_name] = None

        chunk_data.append(chunk_i_dict)
    df = pd.DataFrame(chunk_data)

    model = ### MODEL

    retriever = prepare_multivector_retriever(df, model=model, settings=settings)
    chain = create_multivector_chain(retriever, model)
    # chain = create_chain_with_chain_filter_from_retriever(retriever, settings)

    questions_with_answers_df = pd.read_excel(
        settings.external_data_path / "Questions_20240517.xlsx",
        names=["file_name", "name", "page_num", "question", "answer"],
    )
    questions_with_answers_df["file_name"] = questions_with_answers_df["file_name"].str.normalize(
        "NFD"
    )
    questions_with_answers = questions_with_answers_df["question"].tolist()
    responses = generate_answers(questions_with_answers, chain)
    llm_answers_info = get_llm_answers_info(responses)
    questions_with_answers_df["llm_answer"] = llm_answers_info["llm_answer"]
    questions_with_answers_df["llm_context"] = llm_answers_info["llm_context"]
    # questions_with_answers_df["file_name"] = questions_with_answers_df["file_name"].apply(
    #     lambda file_name: preprocess_file_name(file_name, file_extension="pptx")
    # )

    # questions_with_answers_cols = set(questions_with_answers_df.columns)

    # if "page_num" in questions_with_answers_cols:
    #     if "all_sources" in llm_answers_info:
    #         questions_with_answers_df["reference_chunk_position_in_all_sources"] = (
    #             find_reference_answer_chunk_position(
    #                 questions_with_answers_df, llm_answers_info["all_sources"]
    #             )
    #         )

    #     questions_with_answers_df["reference_chunk_position"] = (
    #         find_reference_answer_chunk_position(
    #             questions_with_answers_df, llm_answers_info["llm_answer_source"]
    #         )
    #     )

    #     questions_with_answers_df["found_reference_chunk"] = questions_with_answers_df[
    #         "reference_chunk_position"
    #     ].apply(answer_has_reference_chunk)

    #     retrieval_accuracy = int(
    #         round(questions_with_answers_df["found_reference_chunk"].mean() * 100)
    #     )
    #     print(f"Retrieval accuracy: {retrieval_accuracy}%")
    #     questions_with_answers_df = questions_with_answers_df.reset_index(drop=True).reindex(
    #         range(len(questions_with_answers_df) + 1)
    #     )

    #     questions_with_answers_df.loc[
    #         len(questions_with_answers_df) - 1, "found_reference_chunk"
    #     ] = f"Retrieval accuracy: {retrieval_accuracy}%"

    answers_file_path = settings.processed_data_path / "answers_for_questions_with_answers.xlsx"
    questions_with_answers_df.to_excel(answers_file_path, index=False)

    # questions_without_answers_df = pd.read_excel(
    #     settings.external_data_path / "Копия Кnigga (002).xlsx",
    #     names=["question", "answer"],
    # )

    # questions_without_answers_df = questions_without_answers_df.dropna()
    # questions_without_answers = questions_without_answers_df["question"].tolist()
    responses = generate_answers(questions_with_answers, chain)
    # questions_without_answers_df["llm_answer"] = [response["answer"] for response in responses]
    # questions_without_answers_df["llm_context"] = [response["context"] for response in responses]
    # answers_file_path = (
        # settings.processed_data_path / "answers_for_questions_without_answers_Кnigga.xlsx"
    # )

    # questions_without_answers_df.to_excel(answers_file_path, index=False)